In [27]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [29]:
import os
print(os.listdir("/gdrive/My Drive/projetum/Banana"))


['overripe', 'ripe', 'rotten', 'unripe']


In [30]:
DATADIR = '/gdrive/My Drive/projetum/Banana'
CATEGORIES=["overripe","ripe","rotten","unripe"]


In [31]:
IMG_SIZE = 100

In [32]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [33]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [34]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [35]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [36]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [37]:
print(y_test)

[1 1 3 ... 1 3 3]


In [38]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(9703, 100, 100, 3)
(2696, 100, 100, 3)
(1079, 100, 100, 3)
(9703,)
(2696,)
(1079,)


In [39]:

import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=4)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=4)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=4)

In [40]:
print(y_test)

[1 1 3 ... 1 3 3]


In [41]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [42]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [43]:
history = model.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 50
                    )

Epoch 1/50
152/152 [==============================] - ETA: 0s - loss: 0.8893 - accuracy: 0.7737
Epoch 1: val_accuracy improved from -inf to 0.45227, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


152/152 [==============================] - 28s 138ms/step - loss: 0.8893 - accuracy: 0.7737 - val_loss: 3.2547 - val_accuracy: 0.4523
Epoch 2/50
152/152 [==============================] - ETA: 0s - loss: 0.3508 - accuracy: 0.8728
Epoch 2: val_accuracy improved from 0.45227 to 0.56163, saving model to model.h5
152/152 [==============================] - 18s 119ms/step - loss: 0.3508 - accuracy: 0.8728 - val_loss: 1.8542 - val_accuracy: 0.5616
Epoch 3/50
152/152 [==============================] - ETA: 0s - loss: 0.2342 - accuracy: 0.9174
Epoch 3: val_accuracy improved from 0.56163 to 0.56905, saving model to model.h5
152/152 [==============================] - 18s 119ms/step - loss: 0.2342 - accuracy: 0.9174 - val_loss: 1.5878 - val_accuracy: 0.5690
Epoch 4/50
152/152 [==============================] - ETA: 0s - loss: 0.1940 - accuracy: 0.9290
Epoch 4: val_accuracy improved from 0.56905 to 0.77201, saving model to model.h5
152/152 [==============================] - 18s 119ms/step - loss: 0

In [44]:
score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

304/304 [==============================] - 5s 14ms/step - loss: 0.2300 - accuracy: 0.9257
Validation Accuracy:  0.9256930947303772
85/85 [==============================] - 1s 16ms/step - loss: 0.3757 - accuracy: 0.8928
Validation Accuracy:  0.8928041458129883
34/34 [==============================] - 1s 26ms/step - loss: 0.4372 - accuracy: 0.8842
Validation Accuracy:  0.8841519951820374


In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test seti üzerinde tahmin yapma
Y_pred = model.predict(x_test, steps=len(x_test))
y_pred = np.argmax(Y_pred, axis=1)

2696/2696 [==============================] - 8s 3ms/step


In [46]:
print(y_pred)

[1 1 3 ... 1 3 3]


In [47]:
# Gerçek etiketleri alma
y_true = y_test

In [48]:
print(y_test)

[1 1 3 ... 1 3 3]


In [49]:
# Accuracy, Precision, Recall, F1 Score hesaplama
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [50]:
# Karmaşıklık Matrisi
conf_matrix = confusion_matrix(y_test, y_pred)

In [51]:

# Classification Report
class_report = classification_report(y_test, y_pred)


In [52]:

# Sonuçları gösterme
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Test Accuracy: 0.8928041543026706
Precision: 0.8964584268240819
Recall: 0.8928041543026706
F1 Score: 0.8920133161865439
Confusion Matrix:
[[448  69  28   0]
 [  3 768  20   0]
 [ 35  49 760  58]
 [  0  17  10 431]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.82      0.87       545
           1       0.85      0.97      0.91       791
           2       0.93      0.84      0.88       902
           3       0.88      0.94      0.91       458

    accuracy                           0.89      2696
   macro avg       0.90      0.89      0.89      2696
weighted avg       0.90      0.89      0.89      2696

